In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reinforcement.players import DummyPlayer, ModelPlayer
from reinforcement.sequential_dqn.dqn_v2 import DQNTrainerV2
from modeling.chessformer import ChessFormer
from torch.optim import Adam
import torch as t
from copy import deepcopy

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BUFFER_SIZE = 1_000_000
RANDOM_ACTION_RATE = 0.0
BATCH_SIZE = 32
NB_STEPS_REWARD = 3
EPSILON_SAMPLING = 0.0001

In [4]:
model_1 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.2).to('cuda')
model_2 = ChessFormer(6, 3, 512, 2048, 64, 8, 0.2).to('cuda')
#model_1_state_dict = t.load('runs/experiment_4/model_1.pt', map_location='cuda')
#model_1.load_state_dict(model_state_dict)
optimizer_1 = Adam(model_1.parameters(), eps=1e-1 ,lr=1e-3)
optimizer_2 = Adam(model_2.parameters(), eps=1e-1, lr=1e-3)
competitor = DummyPlayer()
#competitor_model = deepcopy(model)
#competitor_model.requires_grad_(False)
#competitor = ModelPlayer(competitor_model, random_action_rate=0.0, model_device='cuda')

In [5]:
trainer = DQNTrainerV2(model_1,
                       model_2,
                       optimizer_1,
                       optimizer_2,
                       BUFFER_SIZE,
                       competitor,
                       BATCH_SIZE,
                       experiment_name='experiment_11',
                       models_device='cuda',
                       nb_steps_reward=NB_STEPS_REWARD,
                       epsilon_sampling=EPSILON_SAMPLING)

TypeError: DQNTrainerV2.__init__() got multiple values for argument 'experiment_name'

In [ ]:
trainer.train(10000)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/ubuntu/datalab/ChessFormerRL/reinforcement/sequential_dqn/dqn_v2.py:367: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sampling_probas = t.nn.functional.softmax(sampling_scores).cpu().numpy()
  9%|▉         | 931/10000 [25:46:55<251:08:45, 99.69s/it] 


KeyboardInterrupt: 

In [ ]:
import cProfile

In [ ]:
cProfile.run('trainer.train(1)', 'restats')

  0%|          | 0/1 [00:00<?, ?it/s]/home/ubuntu/datalab/ChessFormerRL/reinforcement/sequential_dqn/dqn_v2.py:367: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sampling_probas = t.nn.functional.softmax(sampling_scores).cpu().numpy()
100%|██████████| 1/1 [02:45<00:00, 165.34s/it]


In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.strip_dirs().sort_stats(-1).print_stats()

Wed Mar 22 16:43:36 2023    restats

         1436762 function calls (1362470 primitive calls) in 165.345 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      246    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.000    0.000  165.347  165.347 <string>:1(<module>)
       46    0.000    0.000    0.000    0.000 <string>:2(__eq__)
    12544    0.005    0.000    0.005    0.000 <string>:2(__init__)
     8979    0.007    0.000    0.013    0.000 _VF.py:26(__getattr__)
      123    0.001    0.000    2.244    0.018 __init__.py:103(backward)
      369    0.021    0.000    0.091    0.000 __init__.py:1223(__str__)
      536    0.001    0.000    0.001    0.000 __init__.py:1410(__init__)
      290    0.000    0.000    0.000    0.000 __init__.py:1430(restore)
        1    0.000    0.000    0.000    0.000 __init__.py:1560(__init__)
      369    0.001    0.000    0.001    0.000 __init__.py:1576(legal_moves)
        1 

In [ ]:
t.save(model_1.state_dict(), 'runs/experiment_11/model.pt')
t.save(model_2.state_dict(), 'runs/experiment_11/model.pt')